In [1]:
# encoding: utf-8
import os
import pandas as pd

In [2]:

def calculate_RFP(l, N, setting = '次数'):
    #计算连续N次测试正确的正确应答个数/出现次数
    i = 0
    a = 0
    for x in l:
        if x == 1:
            i += 1
        else:
            i = 0
            
        if i == N:
            if setting == '应答数':
                a += i
            elif setting == '次数':
                a += 1
        elif i >= N:
            if setting == '应答数':
                a += 1
    return a

def calculate_RP(l):
    #计算坚持性反应数、坚持性错误数
    string = ''.join([str(x) for x in l])
    RP=0
    RPE = 0
    for x in range(len(l))[3:-1]:
        cut = string[x-2:x+2]
        if cut in ['1010','1001','1000','0010','0001','0000','0101','0100']:
            RP += 1
        if cut in ['1001','1000','0001','0000','0101','0100']:
            RPE += 1
    if string[0:3] == '000':
        RP += 3
        RPE += 3
    if string[-3:] in ['100','010','000']:
        RP += 1
        RPE += 1
    return RP, RPE


def wskx(feedback,CC,RF):
    #威斯康星统计计算
    RA = len(feedback)
    RC = sum(feedback)
    RE = RA - RC
    RCP = RC/RA
    l = feedback
    RFP = (calculate_RFP(l,3,'应答数') - (calculate_RFP(l,11)-calculate_RFP(l,12)) - (calculate_RFP(l,12)-calculate_RFP(l,13))*2)/RA
    RP, RPE = calculate_RP(feedback)
    RPEP = RPE/RA
    NRPE = RE - RPE
    FM = calculate_RFP(l,5) - calculate_RFP(l,10)
    return pd.DataFrame({'RA完成测查总应答数':[RA],
                  'CC完成分类数':[CC],
                  'RC正确应答数':[RC],
                  'RE错误应答数':[RE],
                  'RCP正确应答百分比':[RCP],
                  'RF完成第一个分类所需应答数':[RF],
                  'RFP概念化水平百分数':[RFP],
                  'RP持续性应答数':[RP],
                  'RPE持续性错误数':[RPE],
                  'RPEP持续性错误百分数':[RPEP],
                  'NRPE非持续性错误':[NRPE],
                  'FM不能维持完整分类数':[FM]})

In [3]:
import pandas as pd
pd.__version__
df = pd.read_excel('威斯康星测试原始数据_200605.xlsx')[['U_Id','Name','List_Feedback','CC','RF']]
df['input'] = df['List_Feedback'].apply(lambda x: eval(x))
new_data = pd.DataFrame()
for record in df.to_dict(orient='records'):
    data = wskx(record['input'],record['CC'],record['RF'])
    new_data = new_data.append(data)
new_data['U_Id'] = df['U_Id'].tolist()
new_data['Name'] = df['Name'].tolist()
new_data.to_excel('新数据.xlsx')